In [ ]:
import glob
from google.colab import drive
glob.glob('https://docs.google.com/spreadsheets/d/1D7QkDHxUSKnQhR--q0BAwKMxQlUyoJTQ/edit?usp=share_link&ouid=112098211784036012178&rtpof=true&sd=true')

[]

In [ ]:
pip install trafilatura

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 837.8/837.8 kB 55.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 263.8/263.8 kB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.8/293.8 kB 25.7 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import trafilatura
import string
import re
import csv
''' As the file is mounted in my drive, the code successfully gets executed only in my colab '''
''' Or please download the file and copy the path of Input, Output, Stopwords, Negative, Positive txt files '''
df = pd.read_excel(r'/content/drive/MyDrive/BlackCoffer/Input.xlsx') #reading the Input Excel file given
f=df.URL # Storing all the given URL's in a list
number=1
row1 = int(input())
row2 = int(input())
listdataframe=[]
for link in f:
    [positive_score,negative_score,polarity_score,subjectivity_score,percentage_complex_words,fog_index,average_sentence_length,count_complex_words,No_of_words_analysis,Average_syllable,c,average_word_length,No_of_words_after_cleaning,dd]=['','','','','','','','','','','','','','']
    flag=100
    #checking whether URL has content or not
    from urllib.request import urlopen
    from urllib.error import *
    try:
        html = urlopen(link)
    except HTTPError as e:
        flag=1000
    except URLError as e:
        flag=1000
    else:
        active_link=link
    if(flag==100): # As flag doesnt change if link has contents
        downloaded = trafilatura.fetch_url(active_link)
        black=trafilatura.extract(downloaded) #Main content of the webpage is stored as a string in 'black'
        Number_of_sentences=black.count('.') #counting the number of full stops hence number of sentences
        black=black.replace('\n',' ')
        new_string = black.translate(str.maketrans('', '', string.punctuation)) #removing all the punctuations.
        new_string = new_string.replace('“', '')
        new_string = new_string.replace('”', '')
        pattern = r'[0-9]'
        newest_string = re.sub(pattern, '', new_string) #removing all the numbers
        newest_string = newest_string.replace("’s",'')
        all_words=newest_string.split() #converting the string into a list, all words are individual elements of list named 'all_words'
        words_lowered=[]
        for k in all_words:
            words_lowered.append(k.lower()) #lowering every word
        No_of_words_before_cleaning=len(words_lowered)
        with open('/content/drive/MyDrive/BlackCoffer/Stop_Words.txt') as f: #all stopwords given are read
            stop_words_text = f.read() #stop words are stored in a string
        stop_words_final=stop_words_text.split(' ') #converted into a list
        stopwords_final_lowered=[]
        for t in stop_words_final:
            stopwords_final_lowered.append(t.lower()) #all stop words are lowered and stored in a list named "stopwords_final_lowered"
        words_for_analysis=[]
        for g in words_lowered:
            if g not in stopwords_final_lowered: #stop words are removed from our main text
                words_for_analysis.append(g)
        No_of_words_analysis=len(words_for_analysis)
        with open('/content/drive/MyDrive/BlackCoffer/Negative_Words.txt') as g: # reading all the negative words given
            negative_string=g.read() #negative words are stored as a string
        negative_words=negative_string.split(' ') #converted into a list
        with open('/content/drive/MyDrive/BlackCoffer/Positive_Words.txt') as h: # reading all the positive words given
            positive_string=h.read() #positive words are stored as a string
        positive_words=positive_string.split(' ') #converted into a list
        positive_in_ours=[]
        negative_in_ours=[]
        for v in words_for_analysis:
            if v in positive_words:
                positive_in_ours.append(v) #words which are positive are appended
            if v in negative_words:
                negative_in_ours.append(v) #words which are negative are appended
        positive_score=len(positive_in_ours) #calculation of positive score
        negative_score=len(negative_in_ours) #calculation of negative score
        polarity_score=(positive_score-negative_score)/((positive_score+negative_score)+0.000001) #calculation of polarity score
        if(abs(polarity_score)<0.45):
          dd = "Negative"
        elif(abs(polarity_score)>=0.45 and abs(polarity_score)<=0.55):
          dd="Neither Positive nor Negative"
        else:
          dd="Positive"
        subjectivity_score=(positive_score+negative_score)/((No_of_words_analysis)+0.000001) #calculation of subjectivity score
        average_sentence_length=No_of_words_before_cleaning/Number_of_sentences #calculation of Average sentence length i.e average words per sentence
        total_syllable_count=[]
        for word in words_lowered:
            def syllable_count(word):
                count = 0
                vowels = "aeiouy" # vowels are stored in as a string
                if word[0] in vowels:
                    count += 1 #if the first letter of the word is a vowel, then it is counted as a syllable
                for index in range(1, len(word)):
                    if word[index] in vowels and word[index - 1] not in vowels:
                        count += 1 #if a particular letter of a word is a vowel and its previous one isnt a vowel, then it is counted as a syllable
                if word.endswith("e"):
                    count -= 1 #if word ends with "e"
                if (word.endswith("es") and word.endswith("ed")):
                    count -= 1
                if count == 0:
                    count += 1 #no word can have zero syllable
                return count
            count_word=syllable_count(word) #function call and number of syllable is stored in "count_word"
            total_syllable_count.append(count_word) # appended into a list
        count_complex_words=0
        for i in total_syllable_count:
            if i>2: #if a word has more than 2 syllables it is considered to be a complex word
                count_complex_words=count_complex_words+1
        No_of_words_after_cleaning=len(words_lowered)
        Average_syllable = (sum(total_syllable_count)/No_of_words_after_cleaning) #calculating average syllable per word
        percentage_complex_words=count_complex_words/No_of_words_after_cleaning #calculation of percentage of percentage of complex words
        fog_index=0.4*(percentage_complex_words+average_sentence_length) #calculation of fog index
        c=0
        personal_pronouns=['i','we','ours','us','my'] #personal pronouns given
        for k in words_lowered:
            if k in personal_pronouns:
                c=c+1 #number of personal pronouns in the original text before removing stopwords are counted
        count_characters=0
        listaaa=[]
        for k in words_lowered:
            listaaa.append(len(k)) #length of each words are appended into a list named "listaaa"
        count_characters=sum(listaaa) # total number of characters in the original text
        average_word_length=count_characters/No_of_words_before_cleaning ##calculation of Average word length
    listinputs = [number,link,positive_score,negative_score,polarity_score,subjectivity_score,average_sentence_length,percentage_complex_words*100,fog_index,average_sentence_length,count_complex_words,No_of_words_analysis,Average_syllable,c,average_word_length,No_of_words_after_cleaning,dd] #storing all the outputs into a list
    listdataframe.append(listinputs) #appending output list of each webpage into a list


    if(number==row1):
      x = listinputs.copy()
    elif(number==row2):
      y = listinputs.copy()
    number=number+1
# creating data frame
del x[0]
del x[0]
del y[0]
del y[0]
del x[14]
del y[14]
print(x)
print(y)
output_dataframe_copy=pd.DataFrame(listdataframe,columns=['URL ID','URL','POSITIVE SCORE', 'NEGATIVE SCORE', 'POLARITY SCORE','SUBJECTIVITY SCORE','AVG SENTENCE LENGTH','PERCENTAGE OF COMPLEX WORDS','FOG INDEX','AVG NUMBER OF WORDS PER SENTENCE','COMPLEX WORD COUNT','WORD COUNT','SYLLABLE PER WORD','PERSONAL PRONOUNS','AVG WORD LENGTH','WORDCC','RATINGS'])
outputdataframe=pd.DataFrame(listdataframe,columns=['URL ID','URL','POSITIVE SCORE', 'NEGATIVE SCORE', 'POLARITY SCORE','SUBJECTIVITY SCORE','AVG SENTENCE LENGTH','PERCENTAGE OF COMPLEX WORDS','FOG INDEX','AVG NUMBER OF WORDS PER SENTENCE','COMPLEX WORD COUNT','WORD COUNT','SYLLABLE PER WORD','PERSONAL PRONOUNS','AVG WORD LENGTH','WORDCC','RATINGS'])

10
52
[63, 33, 0.31249999674479173, 0.10631229224107167, 25.176470588235293, 19.439252336448597, 10.148345244639913, 25.176470588235293, 416, 903, 1.6948598130841122, 9, 4.941588785046729, 2140]
[31, 11, 0.476190464852608, 0.030589948994471993, 12.582995951417004, 18.693693693693696, 5.107973155341576, 12.582995951417004, 581, 1373, 1.6763191763191763, 7, 5.148648648648648, 3108]


In [ ]:
matrix = np.corrcoef(x, y)
print('Correlation between given two data points : ')
print(matrix)

Correlation between given two data points : 
[[1.         0.99958825]
 [0.99958825 1.        ]]


In [ ]:
file_name = '/content/drive/MyDrive/BlackCoffer/output_black.xlsx' #path of the output file

outputdataframe.to_excel(file_name) #converting the created data frame into an excel file.